# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 23 2022 1:17PM,247166,19,ADV80005907,"AdvaGen Pharma, Ltd",Executing
1,Sep 23 2022 1:17PM,247166,19,ADV80005908,"AdvaGen Pharma, Ltd",Released
2,Sep 23 2022 1:17PM,247166,19,ADV80005909,"AdvaGen Pharma, Ltd",Released
3,Sep 23 2022 1:03PM,247165,10,CTF0011426.1,"Citieffe, Inc.",Released
4,Sep 23 2022 1:03PM,247165,10,CTF0011427,"Citieffe, Inc.",Released
5,Sep 23 2022 12:50PM,247163,16,EMERSA-8563966,"Emersa Waterbox, LLC",Released
6,Sep 23 2022 12:40PM,247162,10,0086005871,ISDIN Corporation,Released
7,Sep 23 2022 12:40PM,247162,10,0086005872,ISDIN Corporation,Released
8,Sep 23 2022 12:40PM,247162,10,0086005881,ISDIN Corporation,Released
9,Sep 23 2022 12:40PM,247162,10,0086005873,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,247162,Released,7
32,247163,Released,1
33,247165,Released,2
34,247166,Executing,1
35,247166,Released,2


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
247161,NaN,NaN,4.0
247162,NaN,NaN,7.0
247163,NaN,NaN,1.0
247165,NaN,NaN,2.0
247166,NaN,1.0,2.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0.0,1.0,0.0
247037,0.0,23.0,11.0
247083,5.0,1.0,4.0
247100,0.0,1.0,0.0
247107,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246891,0,1,0
247037,0,23,11
247083,5,1,4
247100,0,1,0
247107,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,0,1,0
1,247037,0,23,11
2,247083,5,1,4
3,247100,0,1,0
4,247107,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246891,,1,
1,247037,,23,11
2,247083,5,1,4
3,247100,,1,
4,247107,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 23 2022 1:17PM,247166,19,"AdvaGen Pharma, Ltd"
3,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc."
5,Sep 23 2022 12:50PM,247163,16,"Emersa Waterbox, LLC"
6,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation
13,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions
17,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation
26,Sep 23 2022 12:22PM,247159,19,ACG North America LLC
27,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc."
61,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC"
65,Sep 23 2022 12:00PM,247152,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 23 2022 1:17PM,247166,19,"AdvaGen Pharma, Ltd",,1,2
1,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",,,2
2,Sep 23 2022 12:50PM,247163,16,"Emersa Waterbox, LLC",,,1
3,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,,,7
4,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,,,4
5,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,,,9
6,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,,,1
7,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",,23,11
8,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",,2,2
9,Sep 23 2022 12:00PM,247152,10,Emerginnova,,,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:17PM,247166,19,"AdvaGen Pharma, Ltd",2,1,
1,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2,,
2,Sep 23 2022 12:50PM,247163,16,"Emersa Waterbox, LLC",1,,
3,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7,,
4,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4,,
5,Sep 23 2022 12:24PM,247157,10,ISDIN Corporation,9,,
6,Sep 23 2022 12:22PM,247159,19,ACG North America LLC,1,,
7,Sep 23 2022 12:18PM,247037,15,"Person & Covey, Inc.",11,23,
8,Sep 23 2022 12:10PM,247156,15,"Mizner Bioscience, LLC",2,2,
9,Sep 23 2022 12:00PM,247152,10,Emerginnova,2,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:17PM,247166,19,"AdvaGen Pharma, Ltd",2,1,
1,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2,,
2,Sep 23 2022 12:50PM,247163,16,"Emersa Waterbox, LLC",1,,
3,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7,,
4,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:17PM,247166,19,"AdvaGen Pharma, Ltd",2.0,1.0,NaN
1,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2.0,NaN,NaN
2,Sep 23 2022 12:50PM,247163,16,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7.0,NaN,NaN
4,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 23 2022 1:17PM,247166,19,"AdvaGen Pharma, Ltd",2.0,1.0,0.0
1,Sep 23 2022 1:03PM,247165,10,"Citieffe, Inc.",2.0,0.0,0.0
2,Sep 23 2022 12:50PM,247163,16,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,Sep 23 2022 12:40PM,247162,10,ISDIN Corporation,7.0,0.0,0.0
4,Sep 23 2022 12:36PM,247161,16,Sartorius Bioprocess Solutions,4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2965707,74.0,8.0,7.0
15,741310,23.0,32.0,0.0
16,988577,7.0,0.0,0.0
19,988567,4.0,2.0,0.0
21,741122,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2965707,74.0,8.0,7.0
1,15,741310,23.0,32.0,0.0
2,16,988577,7.0,0.0,0.0
3,19,988567,4.0,2.0,0.0
4,21,741122,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,74.0,8.0,7.0
1,15,23.0,32.0,0.0
2,16,7.0,0.0,0.0
3,19,4.0,2.0,0.0
4,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,74.0
1,15,Released,23.0
2,16,Released,7.0
3,19,Released,4.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,21
Status,,,,,
Completed,7.0,0.0,0.0,0.0,0.0
Executing,8.0,32.0,0.0,2.0,2.0
Released,74.0,23.0,7.0,4.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,21
0,Completed,7.0,0.0,0.0,0.0,0.0
1,Executing,8.0,32.0,0.0,2.0,2.0
2,Released,74.0,23.0,7.0,4.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,21
0,Completed,7.0,0.0,0.0,0.0,0.0
1,Executing,8.0,32.0,0.0,2.0,2.0
2,Released,74.0,23.0,7.0,4.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()